In [1]:
# import libraries
import numpy as np
import sys
import psi4
from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)
import json
import git


In [2]:
# get commit number for github repository
repo = git.Repo(search_parent_directories=True)
sha = repo.head.object.hexsha
print(sha)

7402526bbadfcd422647cc60c007497dbe7af540


In [3]:
# setup basic arguments for qed-ci calculation
mol_str = """
Li
H 1 1.8
symmetry c1
"""

options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}

cavity_dict = {
    'omega_value' : 0.12086,
    'lambda_vector' : np.array([0, 0, 0.05]),
    'ci_level' : 'fci',
    'full_diagonalization' : True,
    'number_of_photons' : 10, 
}




In [4]:
# set up base dictionary - some of this will be updated with each calculation
dictionary = {
  "repo" : {
      
      "repository_url" : "https://github.com/mapol-chem/qed-ci",
      "branch" : "photon_number_basis",
      "commit" : sha
  },
    
  "molecule" : {
    "molecule_name": "LiH",
    "geometry": [],
    "symbols": [
      "Li",
      "H"
    ],
    "z-matrix" : mol_str,
  },
    "driver": "energy",
    "model" : {
        "method" : "qed-fci",
        "orbital_basis"  : "sto-3g",
        "photon_basis" : "number_basis",
        "number_of_photon_states" : cavity_dict["number_of_photons"],
        "lambda" : [
            cavity_dict["lambda_vector"][0],
            cavity_dict["lambda_vector"][1],
            cavity_dict["lambda_vector"][2]
        ],
        "omega" : cavity_dict["omega_value"]  
    },
    
    "return_result" : [
        
    ],
    
    "cavity_free_rhf_energy" : 0.,
    "qed_rhf_energy" : 0.,
    "cavity_free_fci_ground_state_energy" : 0.,
}

# function to generate file names based on system details
def generate_file_name(dic, r):
    string_r = "{:.3f}".format(r)
    file_name = dic["molecule"]["molecule_name"] + "_"
    file_name += "r_" + string_r + "_"
    file_name += str(dictionary["model"]["method"]) + "_"
    file_name += str(dictionary["model"]["orbital_basis"]) + "_"
    file_name += str(dictionary["model"]["photon_basis"]) + "_"
    file_name += str(dictionary["model"]["number_of_photon_states"]) + "_"
    file_name += "lambda_" + str(dictionary["model"]["lambda"][0]) 
    file_name += "_" + str(dictionary["model"]["lambda"][1]) 
    file_name += "_" + str(dictionary["model"]["lambda"][2]) + "_"
    file_name += "omega_" + str(dictionary["model"]["omega"]) + ".json"
    return file_name

In [5]:

print(dictionary)
print(generate_file_name(dictionary, 1.833443))

{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': '7402526bbadfcd422647cc60c007497dbe7af540'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.8\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_result': [], 'cavity_free_rhf_energy': 0.0, 'qed_rhf_energy': 0.0, 'cavity_free_fci_ground_state_energy': 0.0}
LiH_r_1.833_qed-fci_sto-3g_number_basis_10_lambda_0.0_0.0_0.05_omega_0.12086.json


In [6]:
mol_tmpl = """
Li
H 1 **R**
symmetry c1
"""

mol_str = """
Li
H 1 1.5
symmetry c1
"""

N_R = 50
N_el = 50
r_array = np.linspace(1.4, 2.2, N_R)

E_array = np.zeros((N_R, N_el))

r_idx = 0
for r in r_array:
    file_name = generate_file_name(dictionary, r) 
    mol_str = mol_tmpl.replace("**R**", str(r))
    mol = psi4.geometry(mol_str)
    #psi4.set_options({'restricted_docc': [3],'active': [4],'num_roots':2})
    scf_e, wfn = psi4.energy('SCF/sto-3g', return_wfn=True)
    fci_energy = psi4.energy('fci',ref_wfn=wfn)    
    LiH = PFHamiltonianGenerator(mol_str, options_dict, cavity_dict)
    
    dictionary["return_result"] = [LiH.CIeigs[0], LiH.CIeigs[1], LiH.CIeigs[2], LiH.CIeigs[3], LiH.CIeigs[4], LiH.CIeigs[5], LiH.CIeigs[6]]
    dictionary["cavity_free_rhf_energy"] = scf_e
    dictionary["cavity_free_fci_ground_state_energy"] = fci_energy
    geometry = np.asarray(mol.geometry())
    geometry_list = geometry.tolist()
    dictionary["molecule"]["geometry"] = geometry_list
    E_array[r_idx, :] = LiH.CIeigs[:50]
    r_idx += 1
    print(dictionary)
    json_object = json.dumps(dictionary, indent=4)
    with open(file_name, "w") as outfile:
        outfile.write(json_object)




Start SCF iterations:

Canonical RHF One-electron energy = -12.6946346236296037
CQED-RHF One-electron energy      = -12.6809948417115983
Nuclear repulsion energy          = 1.1339511657214287
Dipole energy                     = 0.0129085217276915
SCF Iteration   1: Energy = -7.8566740258646472   dE = -7.85667E+00   dRMS = 3.20530E-04
SCF Iteration   2: Energy = -7.8566790098824164   dE = -4.98402E-06   dRMS = 3.95666E-05
SCF Iteration   3: Energy = -7.8566791754252350   dE = -1.65543E-07   dRMS = 1.25930E-05
SCF Iteration   4: Energy = -7.8566792018488094   dE = -2.64236E-08   dRMS = 1.50801E-07
SCF Iteration   5: Energy = -7.8566792018491505   dE = -3.41061E-13   dRMS = 1.03800E-09
SCF Iteration   6: Energy = -7.8566792018491434   dE =  7.10543E-15   dRMS = 1.24962E-10
SCF Iteration   7: Energy = -7.8566792018491434   dE =  0.00000E+00   dRMS = 4.49137E-15
Total time for SCF iterations: 0.001 seconds 

QED-RHF   energy: -7.85667920 hartree
Psi4  SCF energy: -7.86053866 hartree
 Compl

 Completed Hamiltonian build in 0.704373836517334 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': '7402526bbadfcd422647cc60c007497dbe7af540'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.343925708916508], [0.0, 0.0, 2.3942488810325897]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.8\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_result': [-7.877415860453424, -7.759534553176398, -7.75237899103791, -7.7330264151492125, -7.705378220564819, -7.7053782205648185, -7.682796440094305], 'cavity_free_rhf_energy': -7.862539373421326, 'qed_rhf_energy': 0.0, 'cavity_free_fci_ground_state_energy': -7.880942803983375}

Start SCF iterations:

Canonical RHF One-electron energy = -12.6073028869779620
CQED-RHF One-electron energy      = -12.59303

 Completed Hamiltonian build in 0.6934480667114258 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': '7402526bbadfcd422647cc60c007497dbe7af540'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.35942658593809707], [0.0, 0.0, 2.5021586897551855]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.8\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_result': [-7.879000523362043, -7.761824804892719, -7.757463949035951, -7.736874496737777, -7.7090454474869965, -7.709045447486992, -7.687882646046226], 'cavity_free_rhf_energy': -7.863379632324863, 'qed_rhf_energy': 0.0, 'cavity_free_fci_ground_state_energy': -7.882569632578433}

Start SCF iterations:

Canonical RHF One-electron energy = -12.5241587898228453
CQED-RHF One-electron energy      = -12.50

 Completed Hamiltonian build in 0.6772310733795166 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': '7402526bbadfcd422647cc60c007497dbe7af540'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.37492746295968615], [0.0, 0.0, 2.610068498477781]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.8\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_result': [-7.878980463171023, -7.762729527988503, -7.761433793146205, -7.739302336125266, -7.711536807056612, -7.711536807056607, -7.691787377817198], 'cavity_free_rhf_energy': -7.862462462763681, 'qed_rhf_energy': 0.0, 'cavity_free_fci_ground_state_energy': -7.88259512705508}

Start SCF iterations:

Canonical RHF One-electron energy = -12.4449088519790507
CQED-RHF One-electron energy      = -12.42928

 Completed Hamiltonian build in 0.6861591339111328 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': '7402526bbadfcd422647cc60c007497dbe7af540'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.39042833998127524], [0.0, 0.0, 2.7179783072003767]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.8\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_result': [-7.87766112137064, -7.764505817558435, -7.76262592912715, -7.74045069327669, -7.713088051210578, -7.713088051210573, -7.694739023101722], 'cavity_free_rhf_energy': -7.860090615511126, 'qed_rhf_energy': 0.0, 'cavity_free_fci_ground_state_energy': -7.881324414765356}

Start SCF iterations:

Canonical RHF One-electron energy = -12.3692821733550478
CQED-RHF One-electron energy      = -12.352919

...

 Completed Hamiltonian build in 0.6928510665893555 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': '7402526bbadfcd422647cc60c007497dbe7af540'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.4524318480676317], [0.0, 0.0, 3.1496175420907595]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.8\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_result': [-7.863942574083427, -7.770950165469604, -7.757791596992289, -7.734167509521071, -7.713304859838733, -7.713304859838729, -7.700253525137865], 'cavity_free_rhf_energy': -7.840520260362103, 'qed_rhf_energy': 0.0, 'cavity_free_fci_ground_state_energy': -7.867823341946028}

Start SCF iterations:

Canonical RHF One-electron energy = -12.0985141762878659
CQED-RHF One-electron energy      = -12.0788

 Completed Hamiltonian build in 0.6865251064300537 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': '7402526bbadfcd422647cc60c007497dbe7af540'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.4679327250892207], [0.0, 0.0, 3.2575273508133544]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.8\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_result': [-7.859257142551562, -7.771667416391717, -7.756246961251447, -7.730502315183307, -7.712478201051209, -7.712478201051206, -7.7006333417264266], 'cavity_free_rhf_energy': -7.83393096626904, 'qed_rhf_energy': 0.0, 'cavity_free_fci_ground_state_energy': -7.863195346820645}

Start SCF iterations:

Canonical RHF One-electron energy = -12.0378453834023169
CQED-RHF One-electron energy      = -12.0172

 Completed Hamiltonian build in 0.6910300254821777 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': '7402526bbadfcd422647cc60c007497dbe7af540'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.4834336021108098], [0.0, 0.0, 3.3654371595359502]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.8\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_result': [-7.854327834261825, -7.7721894965989495, -7.754649126598873, -7.726327226573857, -7.711474884039246, -7.711474884039243, -7.700775329325206], 'cavity_free_rhf_energy': -7.826906174219507, 'qed_rhf_energy': 0.0, 'cavity_free_fci_ground_state_energy': -7.858323293333054}

Start SCF iterations:

Canonical RHF One-electron energy = -11.9796907659129879
CQED-RHF One-electron energy      = -11.958

 Completed Hamiltonian build in 0.6832098960876465 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': '7402526bbadfcd422647cc60c007497dbe7af540'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.4989344791323988], [0.0, 0.0, 3.473346968258545]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.8\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_result': [-7.849253058981413, -7.772572695831792, -7.752991779744647, -7.72179739725746, -7.710355849929178, -7.710355849929168, -7.700735477916611], 'cavity_free_rhf_energy': -7.819535474592948, 'qed_rhf_energy': 0.0, 'cavity_free_fci_ground_state_energy': -7.85330472554697}

Start SCF iterations:

Canonical RHF One-electron energy = -11.9239294911210791
CQED-RHF One-electron energy      = -11.9013349

 Completed Hamiltonian build in 0.68756103515625 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': '7402526bbadfcd422647cc60c007497dbe7af540'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.5144353561539882], [0.0, 0.0, 3.5812567769811428]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.8\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_result': [-7.844118126044611, -7.772862058344354, -7.751268431398551, -7.717044204547569, -7.709170249453476, -7.709170249453468, -7.700559332564241], 'cavity_free_rhf_energy': -7.811895031041805, 'qed_rhf_energy': 0.0, 'cavity_free_fci_ground_state_energy': -7.848224015871938}

Start SCF iterations:

Canonical RHF One-electron energy = -11.8704521229909634
CQED-RHF One-electron energy      = -11.846809